---

# 8 Puzzle

The 15-puzzle is a sliding tile game that was invented in the 1870s by Noyce Palmer Chapman. We
will focus primarily on its smaller cousin, the 8-puzzle, in this assignment. The game board consists
of eight sliding tiles bearing the numbers 1-8 that move on a 3 × 3 grid. When the puzzle is in the
solved state state, there is a blank space in the top left-hand corner, followed by the tiles in increasing
numerical order from left to right, top to bottom. Any tile horizontally or vertically adjacent to the
blank space can be moved into the space, effectively swapping the position of that tile and the space.
From a scrambled configuration of the tiles, the aim is to repeatedly move appropriate tiles into the
blank space until the tiles and blank square are restored to their solved configuration. This puzzle
easily generalizes to n
2 − 1 tiles, for any natural number n > 3.

In this project, we are coding up 8-puzzle and its natural generalizations to higher dimensions, applying the A* search algorithm with various admissible heuristics.

In [1]:
%pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import any packages you need here
# Also define any variables as needed

import numpy as np
from tabulate import tabulate as tb
from queue import PriorityQueue
import copy

#Now, define the class PuzzleNode:
class PuzzleNode:
    """
    Class PuzzleNode: Provides a structure for performing A* search for the n^2-1 puzzle
    @params:
    puzzle: make our puzzle a numpy array for later purposes such as flattening
    state: the given state of the puzzle which should be list of lists
    dim: dimension of puzzle
    goal_state: goal state of puzzle as numpy
    pruned: boolean whether node in frontier is pruned
    f_val: estimated heursitic value ie. = hval+gval
    g_val: path cost to node from intial state
    parent: parent node
    """
    # YOUR CODE HERE
    def __init__(self, state, f_val = 0, g_val = 0, parent = None):
        self.puzzle = np.array(state)
        self.state = self.puzzle.tolist()
        self.dim = len(state[0])
        self.goal_state = np.arange(len(self.puzzle.flatten())).reshape(self.dim,self.dim)
        self.f_val = f_val
        self.g_val = g_val
        self.pruned = False
        self.parent = parent
    
    def is_valid_puzzle(self):
        """
        Checks if puzzle structure is valid
        return True if valid structure and False otherwise
        """
        if any(len(row) != len(self.state) for row in self.state) or list(range(len(self.state)**2))!=sorted(sum(self.state, [])):
            return False
        if len(self.state) < 3:
            return False
        return True
    
    def empty_slot(self):
        """
        Gets coordinates of empty tile
        returns the coordinates of empty tile
        """
        return np.where(self.puzzle == 0)
    
    def inversions(self):
        """Solves for number of inversions of puzzle.
        Which is useful in determining whether puzzle is solvable or not
        """
        count = 0
        tiles = self.puzzle.flatten()
        for pos, x in enumerate(tiles):
            for y in tiles[pos:]:
                if x > y and y != 0:
                    count += 1
        return count
    
    def is_puzzle_solvable(self):
        """Checks if dim*dim puzzle is solvable at all.
        Here we use the inversions: pair of tiles not in correct ordering.
        These are the rules:
        1. If the puzzle width is odd, then the puzzle will be solvable if number of inversions is even
        2. If puzzle width is even, puzzle is solvable IFF:
        a. number of invertions is odd and row position of the 
            blank tile is even starting from the top.
        b. number of invertions is even and row position of blank tile is odd starting from the top.
        
        returns True if solvable, False otherwise
        """
        total_inversions = self.inversions()
        row = np.where(np.array(self.state) == 0)[0][0]+1
        if len(self.state)%2 != 0 and total_inversions%2 != 0:
            return False
        if len(self.state)%2 == 0 and row%2 != 0 and total_inversions%2 != 0:
            return False
    
        if len(self.state)%2 == 0 and row%2 == 0 and total_inversions%2 == 0:
            return False
        return True
    
    def is_valid_position(self, position):
        """
        param: position- a tuple of x,y coordinate of node to check
        returns: True if given position is a valid tile in puzzle and False otherwise
        """
        if position[0]==self.empty_slot()[0] and position[1]==self.empty_slot()[1]:
            return False #thus position is empty tile
        if position[0] < self.dim and position[0]>= 0:
            if position[1] < self.dim and position[1]>=0:
                return True #means position is not out of puzzle dimensions
        return False
    
    def movable_tiles(self):
        """
        Checks vertical and horizontal neighbors of empty tile
        returns: list of coordinates of adjacent neighbors of empty tile
        """
        movable,next_pos = [],[]
        #right neighbor of empty tile
        next_pos.append((self.empty_slot()[0], self.empty_slot()[1]+1))
        #left neighbor of empty tile
        next_pos.append((self.empty_slot()[0], self.empty_slot()[1]-1))
        #above neighbor of empty tile
        next_pos.append((self.empty_slot()[0]-1, self.empty_slot()[1]))
        #below neighbor of empty tile
        next_pos.append((self.empty_slot()[0]+1, self.empty_slot()[1]))
        for pos in next_pos:
            if self.is_valid_position(pos): #checks if position is valid
                movable.append(pos)
        return movable
    
    def get_next_tile(self, position):
        """
        param position: position tile to swap with empty tile
        returns: tile with new position
        """
        if self.is_valid_position(position):
            puzzle = self.puzzle.copy()
            puzzle[self.empty_slot()],puzzle[position] = puzzle[position],puzzle[self.empty_slot()] #swapping occurs here
            return PuzzleNode(state=puzzle)
        else:
            raise ValueError("Invalid Coordinates")
            
    def find_next_states(self):
        """Similar to method above but this time instead of coordinates, returns all possible next states"""

        zero_index = list(self.empty_slot())
        r, c = (int(i) for i in zero_index)
        n = len(self.state)
        possible_moves = [(r - 1, c), (r + 1, c), (r, c - 1), (r, c + 1)]
        valid_moves = []
        for r, c in possible_moves:
            if 0 <= r < n and 0 <= c < n:
                valid_moves.append([r, c])

        next_states = []
        for move in valid_moves:
            new_state = copy.deepcopy(self.state)
            new_state[zero_index[0][0]][zero_index[1][0]], new_state[move[0]][move[1]] = new_state[move[0]][move[1]], new_state[zero_index[0][0]][zero_index[1][0]]
            next_states.append(new_state)

        return next_states
    
    def is_goal(self):
        return np.all(self.puzzle==self.goal_state)
    
    def __lt__(self, other):
        #used in priority queue for comparison
        return self.f_val < other.f_val
    
    def __str__(self):
        # this would print any/each of the states as n*n grid and should be used if method display_puzzle does not work
        grid = '\n'.join([' '.join([str(col) for col in row]) for row in self.state])
        return '----\n' + grid + '\n----'

    def display_puzzle(self):
        #displays states in a prettier way using tabulate library
        return tb(self.puzzle, tablefmt="fancy_grid")              
        

<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves.

In [3]:
# Add any additional code here (e.g. for the memoization extension)
import os
import json

# YOUR CODE HERE (OPTIONAL)
def cache(h):
    """Memoization to lookup heuristic value for given states already stored"""
    memoization = {}
    
    def helper(state, *args): #args allows us to have varying amounts of parameters
        key = h.__name__
        if key not in memoization:
            memoization[key] = {}
        s_key = str(state)
        if s_key not in memoization:
            memoization[key][s_key] = h(state, *args)
        return memoization[key][s_key]
    return helper

# Misplaced tiles heuristic
@cache
def h1(state):
    """
    This function returns the number of misplaced tiles, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the number of misplaced tiles
    """
    # YOUR CODE HERE
    return sum([1 for pos, x in enumerate(PuzzleNode(state).puzzle.flatten()) if pos!=x and x!=0])

# Manhattan distance heuristic
@cache
def h2(state):
    """
    This function returns the Manhattan distance from the solved state, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Manhattan distance from the solved configuration
    """
    # YOUR CODE HERE
    dist, tile = 0, PuzzleNode(state)
    for x in range(tile.dim):
        for y in range(tile.dim):
            if tile.goal_state[x][y] != 0:
                pos = np.where(tile.puzzle==tile.goal_state[x][y])
                dist += abs(pos[0]-x)+abs(pos[1]-y)
    return int(dist)
    
# Extra heuristic for the extension.  If implemented, modify the function below
@cache
def h4(state, pattern_database_filename = "pattern_database.json"):
    """
    This function returns a heuristic that dominates the Manhattan distance, given the board state
    Input:
        -state: the board state as a list of lists
        -pattern database file: a json file that keeps already explored states and number of steps from that state to goal state.
        e.g {state: steps to goal_state}
    Output:
        -h: the Heuristic distance of the state from its solved configuration
    """
    if os.path.isfile(pattern_database_filename) == False:
        #create new db file if not existing already
        with open(pattern_database_filename,"w") as db_file:
            json.dump({}, db_file)
    patterns = {}
    with open("pattern_database.json","r") as db_file:
        patterns=json.load(db_file)
    
    s_key = str(state)
    if s_key not in patterns:
        #then let's use our extended Manhattan
        return h3(state)
    return patterns[s_key]

@cache
def h3(state, graph_height=3):
    current_heuristic_value = h2(state)
    if current_heuristic_value == 0:
        return 0 #goal states have heuristic values of 0
    if graph_height == 0:
        return current_heuristic_value #if we are at depth 0, then heuristic value is manhattan
    new_heuristic_value = min([h3(child, graph_height-1) for child in PuzzleNode(state).find_next_states()])+1
    return new_heuristic_value #+1 to account for cost of adding child to previous path

# If you implement more than 3 heuristics, then add any extra heuristic functions onto the end of this list.
heuristics = [h1, h2, h4, h3]


<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

In [4]:
# Import any packages or define any helper functions you need here

# YOUR CODE HERE (OPTIONAL)
def get_faulty_puzzle(tile: PuzzleNode):
    """
    Checks if puzzle is valid and solvable
    returns: 0 if no faults, -1 for invalid and -2 for unsolvable
    """
    if tile.is_valid_puzzle() == False:
        return -1
    if tile.is_puzzle_solvable() == False:
        return -2
    return 0


# Main solvePuzzle function.
#This Code is heavily inspired by:
#Title: [A* Tree Search for Robot Navigation]
#Author: [Rohan Shekhar]
#Date: [August, 10, 2017]

def solvePuzzle(state, heuristic, pattern_database_filename = "pattern_database.json", show=False):
    """This function should solve the n**2-1 puzzle for any n > 2 (although it may take too long for n > 4)).
    Inputs:
        -state: The initial state of the puzzle as a list of lists
        -heuristic: a handle to a heuristic function.  Will be one of those defined in Question 2.
        -check_puzzle: test structure for puzzle to see if it is valid and solvable
        -pattern_database_filename: name of database that recognizes patterns
    Outputs:
        -steps: The number of steps to optimally solve the puzzle (excluding the initial state)
        -exp: The number of nodes expanded to reach the solution
        -max_frontier: The maximum size of the frontier over the whole search
        -opt_path: The optimal path as a list of list of lists.  That is, opt_path[:,:,i] should give a list of lists
                    that represents the state of the board at the ith step of the solution.
        -err: An error code.  If state is not of the appropriate size and dimension, return -1.  For the extention task,
          if the state is not solvable, then return -2
    """
    # YOUR CODE HERE
    # Declare Start State and its params
    start_state = PuzzleNode(state)
    steps,exp,max_frontier,optimal_path,err = 0,0,0,None,get_faulty_puzzle(start_state)
    
    #let us do our first check
    if err!=0:
        return steps, exp, max_frontier, optimal_path, err
    
    def h(tile, heuristic=heuristic):
        return heuristic(tile.state)
    current_heuristic = h
    
    start_tile = start_state
    start_tile.f_val = current_heuristic(start_state)
    start_tile.g_val = 0
    
    #hashmap to keep cost to reach visited tiles
    costs_visited = {str(start_state): start_tile}
    
    #Priority Queue to store frontier
    frontier = PriorityQueue()
    frontier.put(start_tile)
    max_frontier += 1
    
    while frontier.empty() == False:
        current_tile: PuzzleNode = frontier.get()
        max_frontier -= 1
        
        if current_tile.pruned == True:
            continue #continue if tile has been pruned
        
        if current_tile.is_goal():
            break #stop and exit loop if we are at goal state
        
        #Get next possible coordinates
        moves = current_tile.movable_tiles()
        for move in moves:
            next_tile = current_tile.get_next_tile(move)
            g_val = current_tile.g_val + 1
            #if nexttile is already in visitedtiles hashmap, we update the path and dont bother exploring again since it is a graph search
            if str(next_tile) in costs_visited:
                if costs_visited[str(next_tile)].g_val > g_val:
                    #mark so that it is removed from queue
                    costs_visited[str(next_tile)].pruned = True
                else:
                    continue #better path already exists
            
            heuristic_value = current_heuristic(next_tile)
            next_tile.f_val = g_val + heuristic_value
            next_tile.g_val = g_val
            next_tile.parent = current_tile
            frontier.put(next_tile) #child tile created and pushed to queue
            max_frontier += 1
            costs_visited[str(next_tile)] = next_tile
        #whenever we get next tile from frontier we have expanded a node
        exp += 1
            
    #Utilizing the pattern database
    patterns = {}
    try:
        with open("pattern_database.json","r") as db_file:
            patterns = json.load(db_file)
    except FileNotFoundError:
        pass
    
    #save optimal steps to goal state from any state to our pattern database
    optimal_path = [current_tile.state]
    with open("pattern_database.json","w") as db_file:
        distance = 0
        while current_tile.parent:
            patterns[str(current_tile.state)] = distance
            optimal_path.append((current_tile.parent).state)
            current_tile = current_tile.parent
            distance += 1
        json.dump(patterns, db_file, indent=4)
    optimal_path = optimal_path[::-1]
    steps = len(optimal_path)-1
    
    if show:
        for tile in optimal_path: 
            print(PuzzleNode(tile).display_puzzle())
    return steps, exp, max_frontier, optimal_path, err

    

<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

In [5]:
working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1])
# Test whether the number of optimal steps is correct and the same
assert(steps_mt == steps_man == 9)
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)
example = solvePuzzle(working_initial_state_15_puzzle, heuristics[1], show=True)
print("\nThe solution requires {} steps".format(example[0]), 
      "with {} expanded nodes".format(example[1]),
      "yeilding a frontier size of {}".format(example[2]), " and error code {}".format(example[4]))

╒═══╤════╤════╤════╕
│ 1 │  2 │  6 │  3 │
├───┼────┼────┼────┤
│ 0 │  9 │  5 │  7 │
├───┼────┼────┼────┤
│ 4 │ 13 │ 10 │ 11 │
├───┼────┼────┼────┤
│ 8 │ 12 │ 14 │ 15 │
╘═══╧════╧════╧════╛
╒═══╤════╤════╤════╕
│ 1 │  2 │  6 │  3 │
├───┼────┼────┼────┤
│ 4 │  9 │  5 │  7 │
├───┼────┼────┼────┤
│ 0 │ 13 │ 10 │ 11 │
├───┼────┼────┼────┤
│ 8 │ 12 │ 14 │ 15 │
╘═══╧════╧════╧════╛
╒═══╤════╤════╤════╕
│ 1 │  2 │  6 │  3 │
├───┼────┼────┼────┤
│ 4 │  9 │  5 │  7 │
├───┼────┼────┼────┤
│ 8 │ 13 │ 10 │ 11 │
├───┼────┼────┼────┤
│ 0 │ 12 │ 14 │ 15 │
╘═══╧════╧════╧════╛
╒════╤════╤════╤════╕
│  1 │  2 │  6 │  3 │
├────┼────┼────┼────┤
│  4 │  9 │  5 │  7 │
├────┼────┼────┼────┤
│  8 │ 13 │ 10 │ 11 │
├────┼────┼────┼────┤
│ 12 │  0 │ 14 │ 15 │
╘════╧════╧════╧════╛
╒════╤════╤════╤════╕
│  1 │  2 │  6 │  3 │
├────┼────┼────┼────┤
│  4 │  9 │  5 │  7 │
├────┼────┼────┼────┤
│  8 │  0 │ 10 │ 11 │
├────┼────┼────┼────┤
│ 12 │ 13 │ 14 │ 15 │
╘════╧════╧════╧════╛
╒════╤════╤════╤════╕
│  1 │  2 │  6 

<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [6]:
## Test for state not correctly defined

incorrect_state = [[0,1,2],[2,3,4],[5,6,7]]
_,_,_,_,err = solvePuzzle(incorrect_state, lambda state: 0)
assert(err == -1)

In [7]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]], [[7,0,8],[4,6,1],[5,3,2]], [[5,7,6],[2,4,3],[8,1,0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7,8,7]
h_man_vals = [15,17,18]

for i in range(0,3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert(h_mt == h_mt_vals[i])
    assert(h_man == h_man_vals[i])


In [8]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [[[2, 3, 7], [1, 8, 0], [6, 5, 4]], [[2, 3, 7], [1, 8, 4], [6, 5, 0]], 
                 [[2, 3, 7], [1, 8, 4], [6, 0, 5]], [[2, 3, 7], [1, 0, 4], [6, 8, 5]], 
                 [[2, 0, 7], [1, 3, 4], [6, 8, 5]], [[0, 2, 7], [1, 3, 4], [6, 8, 5]], 
                 [[1, 2, 7], [0, 3, 4], [6, 8, 5]], [[1, 2, 7], [3, 0, 4], [6, 8, 5]], 
                 [[1, 2, 7], [3, 4, 0], [6, 8, 5]], [[1, 2, 0], [3, 4, 7], [6, 8, 5]], 
                 [[1, 0, 2], [3, 4, 7], [6, 8, 5]], [[1, 4, 2], [3, 0, 7], [6, 8, 5]], 
                 [[1, 4, 2], [3, 7, 0], [6, 8, 5]], [[1, 4, 2], [3, 7, 5], [6, 8, 0]], 
                 [[1, 4, 2], [3, 7, 5], [6, 0, 8]], [[1, 4, 2], [3, 0, 5], [6, 7, 8]], 
                 [[1, 0, 2], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

astar_steps = [17, 25, 28]
for i in range(0,3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[0])
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_mt == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert(expansions_man < expansions_mt)
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert(opt_path_mt == opt_path_soln)


In [9]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1])
# Test whether the number of optimal steps is correct and the same
assert(steps_mt == steps_man == 9)
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)

<h1>Extension Tests</h1>
The cells below can be used to test the extension questions.  Memoization if implemented will be tested on the final submission - you can test it yourself by testing the execution time of the heuristic functions with and without it.

In [10]:
## Puzzle solvability test

unsolvable_initial_state = [[7,5,6],[2,4,3],[8,1,0]]
_,_,_,_,err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert(err == -2)

In [11]:
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]], [[7,0,8],[4,6,1],[5,3,2]], [[5,7,6],[2,4,3],[8,1,0]])
dom = 0
for i in range(0,3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[2])
    steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_new == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    print("Manhattan expansions: ", expansions_man, " New Pattern Expansions: ", expansions_new)
    dom = expansions_man - expansions_new
    print(dom > 0)

Manhattan expansions:  114  New Pattern Expansions:  70
True
Manhattan expansions:  2104  New Pattern Expansions:  671
True
Manhattan expansions:  1582  New Pattern Expansions:  1074
True


In [12]:
## Extra heuristic function test.  
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.

dom = 0
for i in range(0,3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[2])
    steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_new == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
    assert(dom > 0)